This jyupter note book is for running the defined funcions in pdbdownload

Goal is to extract pdb files based on a selection of ID from a csv
subset the csv for the selected ID and remove the ids that have multiple antigen chains separated by  | .




In [1]:
from Bio import PDB
from Bio.PDB import PDBParser, PDBIO
from Bio import SeqIO
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pdbdownload as pdbdl

In [3]:
main_csv = pd.read_csv("HOMOE_Ab_An_summary.csv")

print(main_csv.head())

print("Number of rows in main_csv:", main_csv.shape[0])

    pdb Hchain Lchain  model antigen_chain antigen_type antigen_het_name  \
0  9fys      B      C      0             X      protein              NaN   
1  9fys      I      M      0             Z      protein              NaN   
2  9fyt      I      M      0             B      protein              NaN   
3  9fyt      C      D      0             A      protein              NaN   
4  9uxd      D      E      0             A      protein              NaN   

         antigen_name                 short_header       date  ...   scfv  \
0  alpha-bungarotoxin                        TOXIN  7/16/2025  ...  False   
1  alpha-bungarotoxin                        TOXIN  7/16/2025  ...  False   
2    alpha-cobratoxin                        TOXIN  7/16/2025  ...  False   
3    alpha-cobratoxin                        TOXIN  7/16/2025  ...  False   
4  spike glycoprotein  VIRAL PROTEIN/IMMUNE SYSTEM  7/16/2025  ...  False   

  engineered heavy_subclass light_subclass light_ctype affinity delta_g  \
0    

In [4]:
import pdbdownload as pdbdl
#subset the PDB ID for first 2000
subset_df = pdbdl.subset_pdb_ids("HOMOE_Ab_An_summary.csv")


In [5]:
filtered_df = pdbdl.filter_pdb_ids("subset_pdb_ids.csv")


Original rows: 2000
After removing NaN values: 1919
After removing rows with '|' in antigen_chain: 0
After keeping first occurrence of each PDB: 0
Saved 0 unique PDB entries to filtered_pdb_ids.csv


In [6]:
# Debug: Let's check what's in the subset file and why filtering results in empty CSV
import pandas as pd

# Load the subset file to see what we're working with
subset_df = pd.read_csv("subset_pdb_ids.csv")
print(f"Subset file has {len(subset_df)} rows")
print(f"Columns: {list(subset_df.columns)}")
print("\nFirst few rows:")
print(subset_df.head())

# Check for missing values in key columns
print(f"\nMissing values:")
print(f"pdb: {subset_df['pdb'].isna().sum()}")
print(f"antigen_chain: {subset_df['antigen_chain'].isna().sum()}")
print(f"Hchain: {subset_df['Hchain'].isna().sum()}")
print(f"Lchain: {subset_df['Lchain'].isna().sum()}")

# Check how many rows would remain after dropna
after_dropna = subset_df.dropna(subset=['pdb', 'antigen_chain', 'Hchain', 'Lchain'])
print(f"\nRows after removing NaN: {len(after_dropna)}")

# Check how many have "|" in antigen_chain
if len(after_dropna) > 0:
    has_pipe = after_dropna['antigen_chain'].str.contains('|', na=False).sum()
    print(f"Rows with '|' in antigen_chain: {has_pipe}")
    print(f"Rows without '|' in antigen_chain: {len(after_dropna) - has_pipe}")
    
    # Show some examples of antigen_chain values
    print(f"\nSample antigen_chain values:")
    print(after_dropna['antigen_chain'].head(10).tolist())

Subset file has 2000 rows
Columns: ['pdb', 'Hchain', 'Lchain', 'model', 'antigen_chain', 'antigen_type', 'antigen_het_name', 'antigen_name', 'short_header', 'date', 'compound', 'organism', 'heavy_species', 'light_species', 'antigen_species', 'authors', 'resolution', 'method', 'r_free', 'r_factor', 'scfv', 'engineered', 'heavy_subclass', 'light_subclass', 'light_ctype', 'affinity', 'delta_g', 'affinity_method', 'temperature', 'pmid']

First few rows:
    pdb Hchain Lchain  model antigen_chain antigen_type antigen_het_name  \
0  9fys      B      C      0             X      protein              NaN   
1  9fys      I      M      0             Z      protein              NaN   
2  9fyt      I      M      0             B      protein              NaN   
3  9fyt      C      D      0             A      protein              NaN   
4  9uxd      D      E      0             A      protein              NaN   

         antigen_name                 short_header       date  ...   scfv  \
0  alpha-bun

In [7]:
# Test the pipe character detection
after_dropna = subset_df.dropna(subset=['pdb', 'antigen_chain', 'Hchain', 'Lchain'])

# Test different ways to detect "|"
print("Testing pipe detection methods:")
print(f"Method 1 - str.contains('|'): {after_dropna['antigen_chain'].str.contains('|', na=False).sum()}")
print(f"Method 2 - str.contains('\\|'): {after_dropna['antigen_chain'].str.contains('\\|', na=False).sum()}")
print(f"Method 3 - str.contains('|', regex=False): {after_dropna['antigen_chain'].str.contains('|', regex=False, na=False).sum()}")

# Check if any antigen_chain actually contains "|"
actual_pipes = after_dropna[after_dropna['antigen_chain'].str.contains('\\|', na=False)]
print(f"\nRows that actually contain '|': {len(actual_pipes)}")

if len(actual_pipes) > 0:
    print("Examples with '|':")
    print(actual_pipes['antigen_chain'].head())
else:
    print("No rows actually contain '|' character")

Testing pipe detection methods:
Method 1 - str.contains('|'): 1919
Method 2 - str.contains('\|'): 363
Method 3 - str.contains('|', regex=False): 363

Rows that actually contain '|': 363
Examples with '|':
26    B | A
31    C | D
36    B | A
43    B | A
67    A | B
Name: antigen_chain, dtype: object


In [8]:
# Test the fixed filtering function
import importlib
importlib.reload(pdbdl)  # Reload the module to get the updated function

filtered_df = pdbdl.filter_pdb_ids("subset_pdb_ids.csv")

Original rows: 2000
After removing NaN values: 1919
After removing rows with '|' in antigen_chain: 1556
After keeping first occurrence of each PDB: 698
Saved 698 unique PDB entries to filtered_pdb_ids.csv


In [ ]:
# Reload the module after installing requests
import importlib
import pdbdownload as pdbdl
importlib.reload(pdbdl)

# Load the filtered CSV file as a DataFrame
filtered_df = pd.read_csv("filtered_pdb_ids.csv")
print(f"Loaded {len(filtered_df)} filtered PDB entries")

# Extract PDB IDs and set up download
pdb_ids = filtered_df['pdb'].tolist()
print(f"Will download {len(pdb_ids)} PDB files")

save_directory = "D:/VIDO/VIDO PROJECT FILES/PDB_legacy_files"

#download the PDB legacy files for the selected PDB ids
pdbdl.download_pdb_files(pdb_ids, save_directory)

Loaded 698 filtered PDB entries
Will download 698 PDB files


NameError: name 'requests' is not defined